In [85]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib
#matplotlib.use('cairo.png')
try:
    get_ipython().run_line_magic("matplotlib inline")
except:
    plt.ion()
    
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import xml.etree.ElementTree as ET
from Model import StenosisDetector
from AngioDataset import MyDataset
from utils import *
import torchvision
from matplotlib.patches import Rectangle

ImportError: cannot import name 'TryExcept'

In [64]:
#load the data
img_dir = 'sample_data/image_dir'
xml_dir = 'sample_data/xml_dir'
dataset = MyDataset(img_dir=img_dir, xml_dir=xml_dir)
angio_data = torch.utils.data.DataLoader(dataset,batch_size=1)

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [82]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /home/ubuntu/.cache/torch/hub/master.zip


ImportError: cannot import name 'TryExcept'

In [70]:
for (idx, batch) in enumerate(angio_data):
    print(batch[1][0,:,:,:].shape)
    plt.imshow(batch[1][0,:,:,:])
    plt.savefig('image_storage/testx.png')

torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])


In [48]:
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        # Define the encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU()
        )
        # Define the latent representation
        self.fc_mu = nn.Linear(16, latent_dim)
        self.fc_logvar = nn.Linear(16, latent_dim)

        # Define the decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        z = self.reparameterize(mu, logvar)
        reconstructed = self.decoder(z)
        return reconstructed, mu, logvar

    def reparameterize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        eps = torch.randn_like(std)
        return mu + std*eps

# Train the VAE on the normal data
vae = VAE(input_dim=512, latent_dim=10)

In [79]:
data = torch.randn(512, 512, 1)
data.shape
#data = angio_data

torch.Size([512, 512, 1])

In [80]:
optimizer = torch.optim.Adam(vae.parameters())
model = VAE(input_dim=512, latent_dim=10)


loss_fn = nn.BCELoss()

# Train the model
for epoch in range(100):

    # Compute the reconstruction loss
    reconstructed, mu, logvar = model(data)
    reconstruction_loss = loss_fn(reconstructed, data)
    
    # Compute the KL divergence loss
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    # Compute the total loss
    total_loss = reconstruction_loss + kl_loss
    
    # Backpropagate the gradients and update the model weights
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    # Print the loss values
    #print(f"Epoch {epoch}: reconstruction_loss = {reconstruction_loss:.4f}, kl_loss = {kl_loss:.4f}, total_loss = {total_loss:.4f}")
    print(reconstructed)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (262144x1 and 512x32)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class DarknetConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(DarknetConv, self).__init__()
        self.conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias=False,
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.leaky_relu = nn.LeakyReLU(0.1)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.leaky_relu(x)
        return x


class DarknetBlock(nn.Module):
    def __init__(self, in_channels):
        super(DarknetBlock, self).__init__()
        self.conv1 = DarknetConv(in_channels, in_channels // 2, 1, 1, 0)
        self.conv2 = DarknetConv(in_channels // 2, in_channels, 3, 1, 1)

    def forward(self, x):
        residual = x
        x = self.conv1(x)
        x = self.conv2(x)
        x = x + residual
        return x


class YOLOv3(nn.Module):
    def __init__(self, num_classes=80, num_anchors=3):
        super(YOLOv3, self).__init__()
        self.num_classes = num_classes
        self.num_anchors = num_anchors

        # Backbone
        self.conv1 = DarknetConv(3, 32, 3, 1, 1)
        self.conv2 = DarknetConv(32, 64, 3, 2, 1)
        self.residual_block1 = self._make_residual_block(64, 1)
        self.conv3 = DarknetConv(64, 128, 3, 2, 1)
        self.residual_block2 = self._make_residual_block(128, 2)
        self.conv4 = DarknetConv(128, 256, 3, 2, 1)
        self.residual_block3 = self._make_residual_block(256, 8)
        self.conv5 = DarknetConv(256, 512, 3, 2, 1)
        self.residual_block4 = self._make_residual_block(512, 8)
        self.conv6 = DarknetConv(512, 1024, 3, 2, 1)
        self.residual_block5 = self._make_residual_block(1024, 4)

        # Detection layers
        self.detection_layers = nn.ModuleList([
            DarknetConv(1024, 512, 1, 1, 0),
            DarknetConv(512, 1024, 3, 1, 1),
            DarknetConv(1024, 512, 1, 1, 0),
            DarknetConv(512, 1024, 3, 1, 1),
            DarknetConv(1024, 512, 1, 1, 0),
        ])

        self.prediction_layers = nn.ModuleList([
            nn.Conv2d(512, num_anchors * (num_classes + 5), 1, 1, 0),
            nn.Conv2d(1024, num_anchors * (num_classes + 5), 1, 1, 0),
            nn.Conv2d(512, num_anchors * (num_classes + 5), 1, 1, 0),
            nn.Conv2d(1024, num_anchors * (num_classes + 5), 1, 1, 0),
            nn.Conv2d(512, num_anchors * (num_classes + 5), 1, 1, 0),
        ])

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.residual_block1(x)
        x = self.conv3(x)
        x = self.residual_block2(x)
        x = self.conv4(x)
        x = self.residual_block3(x)
        x = self.conv5(x)
        x = self.residual_block4(x)
        x = self.conv6(x)
        x = self.residual_block5(x)

        output = []
        for i in range(5):
            x = self.detection_layers[i](x)
            pred = self.prediction_layers[i](x)
            output.append(pred)

            if i < 3:
                x = self._upsample(x)

        return output

    def _make_residual_block(self, in_channels, num_blocks):
        layers = []
        for _ in range(num_blocks):
            layers.append(DarknetBlock(in_channels))
        return nn.Sequential(*layers)


class YOLOLoss(nn.Module):
    def __init__(self, num_classes=80, num_anchors=3):
        super(YOLOLoss, self).__init__()
        self.num_classes = num_classes
        self.num_anchors = num_anchors
        self.bce_loss = nn.BCEWithLogitsLoss()
        self.mse_loss = nn.MSELoss()

    def forward(self, output, targets):
        device = output[0].device
        loss = torch.tensor(0.0, device=device)

        for i in range(len(output)):
            pred = output[i]
            batch_size, _, grid_size, _ = pred.shape
            stride = 416 // grid_size

            obj_mask = targets[i][:, :, 4:5].bool()
            noobj_mask = ~obj_mask
            class_labels = targets[i][:, :, 5:]
            true_box = targets[i][:, :, :4]

            pred[:, :, :, 0:2] = torch.sigmoid(pred[:, :, :, 0:2])
            pred[:, :, :, 2:4] = torch.exp(pred[:, :, :, 2:4]) * stride

            iou = bbox_iou(pred[..., :4], true_box)

            loc_loss = torch.sum(obj_mask * (1 - iou) ** 2)

            conf_pos_loss = torch.sum(obj_mask * self.bce_loss(pred[..., 4:5], obj_mask.float()))
            conf_neg_loss = torch.sum(noobj_mask * self.bce_loss(pred[..., 4:5], obj_mask.float()))
            conf_loss = conf_pos_loss + conf_neg_loss

            class_loss = torch.sum(obj_mask * self.bce_loss(pred[..., 5:], class_labels.float()))

            layer_loss = loc_loss + conf_loss + class_loss
            loss += layer_loss

        return loss


def bbox_iou(box1, box2):
    """
    Calculate IoU (Intersection over Union) between two sets of bounding boxes.
    Each bounding box is represented as (x, y, w, h), where x, y are the coordinates
    of the top-left corner, and w, h are the width and height respectively.
    Args:
        box1: (Tensor) bounding boxes, Shape: [N, 4]
        box2: (Tensor) bounding boxes, Shape: [M, 4]
    Returns:
        iou: (Tensor) Shape: [N, M]
    """
    N = box1.size(0)
    M = box2.size(0)

    # Unpack coordinates of the boxes
    x1, y1, w1, h1 = box1[:, 0], box1[:, 1], box1[:, 2], box1[:, 3]
    x2, y2, w2, h2 = box2[:, 0], box2[:, 1], box2[:, 2], box2[:, 3]

    # Calculate coordinates of the intersection rectangle
    intersection_x1 = torch.max(x1 - w1 / 2, x2 - w2 / 2)
    intersection_y1 = torch.max(y1 - h1 / 2, y2 - h2 / 2)
    intersection_x2 = torch.min(x1 + w1 / 2, x2 + w2 / 2)
    intersection_y2 = torch.min(y1 + h1 / 2, y2 + h2 / 2)

    # Calculate the area of intersection rectangle
    intersection_area = torch.clamp(intersection_x2 - intersection_x1 + 1, min=0) * torch.clamp(
        intersection_y2 - intersection_y1 + 1, min=0
    )

    # Calculate the area of both bounding boxes
    box1_area = w1 * h1
    box2_area = w2 * h2

    # Calculate IoU
    iou = intersection_area / (box1_area + box2_area - intersection_area + 1e-16)

    return iou


# Usage example
model = YOLOv3(num_classes=80, num_anchors=3)
loss_fn = YOLOLoss(num_classes=80, num_anchors=3)

# Generate example input and target tensors
input_tensor = torch.randn(2, 3, 416, 416)
target_tensor = torch.randn(2, 5, 13, 13)

# Forward pass
output = model(input_tensor)

# Calculate loss
loss = loss_fn(output, target_tensor)

# Print the loss
print(loss)

# Perform predictions
# ...
